In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
##need to encode labels for xgboost
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import lightgbm as lgb

In [ ]:
## Test 3: 0.79268
## Test 4: same results
## test 5: 0.76829 -- using categorical variables
## Test 7: 0.80487 After making Hospital Categorical.  Will need to lookinto removing a column or two.
## Test 8 and 9:  Removing all lesions was a step backwards.  I will try just removing one or two.
## Test 10/11 produced close to same results as 8 and 9


In [ ]:
test_file = '/kaggle/input/playground-series-s3e22/test.csv'
train_file = '/kaggle/input/playground-series-s3e22/train.csv'

In [ ]:
test_data = pd.read_csv(test_file)
train_data = pd.read_csv(train_file)

In [ ]:
train_describe = train_data.copy()
test_describe = test_data.copy()

In [ ]:
train_data.describe().drop(['id','hospital_number'],axis=1)

In [ ]:
test_describe.describe().drop(['id','hospital_number'],axis=1)

### This cell was created after the 7th test.  Wanted to know the correlation between columns

In [ ]:
dummies_train = train_data.select_dtypes(include='object')
dummies_test = test_data.select_dtypes(include='object')

d_train = pd.get_dummies(train_data, columns=dummies_train.columns)
d_test = pd.get_dummies(test_data, columns=dummies_test.columns)

##not sure if hospital_number will through off the correlation.
##I removed the columns that were missing in or the other dataframe.  See cells before calculation
##Next I removed lesions because they are closest to white and there are two correlation columns for lesion_surgery.  I will rerun the calculations without lesions
##Test 8/9 are trash.  Will start trying with 3.
d_train = d_train.drop(['id','hospital_number','nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 
                        'peristalsis_distend_small', 'pain_slight','lesion_3'],axis=1)
d_test = d_test.drop(['id','hospital_number'],axis=1)

In [ ]:
d_train.corr(method='pearson').style.background_gradient(cmap='Blues')

In [ ]:
##Adding missing data
##EA was done as part of another notebook.  Will try and keep the rest of the notebooks clean.
for column in train_data.select_dtypes(include=["object"]):
    train_data[column]=train_data[column].fillna("no_measure_recorded")
    
for column in test_data.select_dtypes(include=["object"]):
    test_data[column]=test_data[column].fillna("no_measure_recorded")
    
for column in train_data.select_dtypes(include=["int64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["int64"]):
    test_data[column]=test_data[column].fillna(0)

for column in train_data.select_dtypes(include=["float64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["float64"]):
    test_data[column]=test_data[column].fillna(0)

In [ ]:
def remove_missing_columns(df,column_set):
    for column in column_set:
        if column in df.columns:
            df = df.drop(columns=column)
    return df        
    

missing_columns={'nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 'peristalsis_distend_small', 'pain_slight','pain_moderate',"lesion_3"}

test_data = remove_missing_columns(test_data,missing_columns)
train_data = remove_missing_columns(train_data,missing_columns)

# test_data = test_data.drop(columns=missing_columns)
# train_data = train_data.drop(columns=missing_columns)


In [ ]:
le = LabelEncoder()
train_data['encoded_outcome'] = le.fit_transform(train_data['outcome'])
y = train_data['encoded_outcome']
train_data.head()
test_data.head()
y.shape

In [ ]:
trained_dropped = train_data.drop(['id','encoded_outcome','outcome'],axis=1) ## Tried with and without hospital.
test_dropped = test_data.drop(['id'], axis=1)


trainedc = trained_dropped.copy()
testc = test_dropped.copy()
trained_dropped.shape,test_dropped.shape

In [ ]:
## Didn't realize I needed to turn things into dummies
train_dummy_objects = trained_dropped.select_dtypes(include='object')
test_dummy_objects = test_dropped.select_dtypes(include='object')
print(train_dummy_objects.shape,test_dummy_objects.shape
)
test_with_dummies = pd.get_dummies(test_dropped, columns=test_dummy_objects.columns)
train_with_dummies = pd.get_dummies(trained_dropped, columns=train_dummy_objects.columns)
print(test_with_dummies.shape,train_with_dummies.shape)

#category
trained_cats = trainedc.select_dtypes(include=['object']).columns
test_cats = testc.select_dtypes(include=['object']).columns

trainedc[trained_cats] = trainedc[trained_cats].astype('category')
testc[test_cats] = testc[test_cats].astype('category')

trainedc['hospital_number'] = trainedc['hospital_number'].astype('category')
testc['hospital_number'] = testc['hospital_number'].astype('category')



print(trainedc.shape,testc.shape)

In [ ]:
def remove_missing_columns(df,column_set):
    for column in column_set:
        if column in df.columns:
            df = df.drop(columns=column)
    return df        
    

missing_columns={'nasogastric_reflux_slight', 'rectal_exam_feces_serosanguious', 'peristalsis_distend_small', 
                 'pain_slight','pain_moderate','lesion_3'}#'lesion_1','lesion_2','lesion_3'

test_with_dummies = remove_missing_columns(test_with_dummies,missing_columns)
train_with_dummies = remove_missing_columns(train_with_dummies,missing_columns)

testc = remove_missing_columns(testc,missing_columns)
trainedc = remove_missing_columns(trainedc,missing_columns)



In [ ]:
X = train_with_dummies
X.shape
Xcat = trainedc
Xcat.shape

In [ ]:
X.dtypes

In [ ]:
xgb_model = xgb.XGBClassifier(learning_rate=0.1, n_estimator=1000, max_depth=3,
                              objective='binary:logistic',random_state=42)
xgb_model.fit(X,y)
xgb_model_cat = xgb.XGBClassifier(tree_method="hist", enable_categorical=True,device="cuda")
xgb_model_cat.fit(Xcat,y)

In [ ]:
test_with_dummies.shape

In [ ]:
train_with_dummies.shape

In [ ]:
y_xgb_pred = xgb_model.predict(test_with_dummies)
y_xgb_pred_cat = xgb_model_cat.predict(testc)

In [ ]:
# y_xgb_pred_cat
# y_xgb_pred

In [ ]:
labels = ['died', 'euthanized', 'lived']

leII = LabelEncoder()
leII.fit(labels)

encoded_label = leII.transform(labels)

ypred_labels = leII.inverse_transform(y_xgb_pred)
ypred_cat_labels = leII.inverse_transform(y_xgb_pred_cat)


In [ ]:
ypcat_labels = test_data.copy()

ypcat_labels['outcome'] = ypred_cat_labels
test_data['outcome'] = ypred_labels

In [ ]:
output_y = test_data[['id','outcome']]
output_y.to_csv('outputXII.csv',index=False)

output_ycat = ypcat_labels[['id','outcome']]
output_ycat.to_csv('outputXIII.csv',index=False)
